### 영화 리뷰에 따라 tag를 10개 정도 추출하는 모델 제작


In [10]:
#keybert 테스트
from keybert import KeyBERT

kw_model = KeyBERT()

reviews = [
    '너무 진지', 
    '무대 연출', 
    '진지해요 여자',
    '연출과 가창력', 
    '연주자의 소리', 
    '기반으로 연출', 
    '볼만합니다', 
    '피가로역 연주자', 
    '독특하구요 탄탄한'
    ]


full_text = " ".join(reviews)
keywords = kw_model.extract_keywords(full_text, top_n=30, keyphrase_ngram_range=(1, 2))

print([kw[0] for kw in keywords])


['가창력 연주자의', '연출 진지해요', '연출과 가창력', '연주자 독특하구요', '기반으로 연출', '여자 연출과', '피가로역 연주자', '연출 볼만합니다', '독특하구요 탄탄한', '볼만합니다 피가로역', '연주자의 소리', '진지해요 여자', '무대 연출', '연출과', '연주자의', '진지해요', '연출', '독특하구요', '연주자', '가창력', '너무 진지', '볼만합니다', '진지 무대', '피가로역', '소리 기반으로', '여자', '기반으로', '탄탄한', '진지', '소리']


In [13]:
keywords = kw_model.extract_keywords(full_text, top_n=7, keyphrase_ngram_range=(1, 1))


In [14]:
print([kw[0] for kw in keywords])

['연출과', '연주자의', '진지해요', '연출', '독특하구요', '연주자', '가창력']


In [1]:
import pandas as pd
from keybert import KeyBERT
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer

# 데이터 로드
df = pd.read_csv("1.megabox_reviews_merged_avg.csv")
review_cols = ["리뷰1", "리뷰2", "리뷰3", "리뷰4", "리뷰5"]
df["전체리뷰"] = df[review_cols].fillna("").agg(" ".join, axis=1)

# 분석기 초기화
kw_model = KeyBERT()
okt = Okt()

# 키워드 추출 함수
def extract_clean_keywords(text):
    try:
        keywords = kw_model.extract_keywords(text, top_n=10, keyphrase_ngram_range=(1, 2))
        phrases = [kw[0] for kw in keywords]

        refined = []
        for phrase in phrases:
            tokens = okt.pos(phrase, stem=True)
            for word, tag in tokens:
                if tag in ["Noun", "Adjective", "Verb"] and len(word) > 1:
                    refined.append(word)
        return " ".join(refined)
    except:
        return ""

# 정제 키워드 문장 생성
df["정제단어문장"] = df["전체리뷰"].apply(extract_clean_keywords)

# TF-IDF 기반 키워드 추출
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df["정제단어문장"])
feature_names = vectorizer.get_feature_names_out()

def get_top_tfidf_keywords(row_idx, top_n=5):
    row = tfidf_matrix[row_idx]
    scores = row.toarray().flatten()
    top_indices = scores.argsort()[-top_n:][::-1]
    return ", ".join([feature_names[i] for i in top_indices])

df["최종키워드"] = [get_top_tfidf_keywords(i) for i in range(len(df))]

# 저장
df.to_csv("megabox_final_keywords_tfidf.csv", index=False, encoding="utf-8-sig")
print("🎉 최종 키워드 추출 완료 → megabox_final_keywords_tfidf.csv")


f:\term_pro\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



🎉 최종 키워드 추출 완료 → megabox_final_keywords_tfidf.csv
